## Define test data constants & distance formula

In [10]:
import numpy as np
import random
import math

depot = (4.4184, 114.0932)
customers = [
    (4.3555, 113.9777, 5),
    (4.3976, 114.0049, 8),
    (4.3163, 114.0764, 3),
    (4.3184, 113.9932, 6),
    (4.4024, 113.9896, 5),
    (4.4142, 114.0127, 8),
    (4.4804, 114.0734, 3),
    (4.3818, 114.2034, 6),
    (4.4935, 114.1828, 5),
    (4.4932, 114.1322, 8)
]

vehicle_types = {
    'A': {'capacity': 25, 'cost_per_km': 1.2},
    'B': {'capacity': 30, 'cost_per_km': 1.5}
}

def calculate_distance(loc1, loc2):
    return 100 * math.sqrt((loc2[1] - loc1[1])**2 + (loc2[0] - loc1[0])**2)

# Generates an initial population of solutions represent a set of routes for the vehicles.

In [11]:
# Generate initial population
def generate_initial_population(pop_size, num_vehicles):
    population = []
    for _ in range(pop_size):
        unassigned_customers = customers.copy()  # Start with all unassigned customers
        random.shuffle(unassigned_customers)
        routes = []
        for i in range(num_vehicles):
            if not unassigned_customers:
                break
            vehicle_type = random.choice(['A', 'B'])
            route_capacity = vehicle_types[vehicle_type]['capacity']
            route = [depot]
            current_capacity = 0
            assigned_indices = []
            for idx, customer in enumerate(unassigned_customers):
                if current_capacity + customer[2] <= route_capacity:
                    route.append(customer)
                    current_capacity += customer[2]
                    assigned_indices.append(idx)
                if current_capacity >= route_capacity:
                    break
            for idx in sorted(assigned_indices, reverse=True):
                unassigned_customers.pop(idx)
            route.append(depot)
            routes.append((vehicle_type, route))
        population.append(routes)
    return population

# Calculates the fitness value of a given set of routes in terms of total costs

In [12]:
def fitness_function(routes):
    total_cost = 0
    for vehicle_type, route in routes:
        route_cost = 0
        for i in range(len(route) - 1):
            distance = calculate_distance(route[i], route[i+1])
            route_cost += distance
        total_cost += route_cost * vehicle_types[vehicle_type]['cost_per_km']
    return total_cost

# Performs one-point crossover between two parent solutions to create offspring.

In [13]:
def one_point_crossover(parent1, parent2):
    if len(parent1) > 1 and len(parent2) > 1:
        crossover_point = random.randint(1, min(len(parent1), len(parent2)) - 1)
        child1_routes = parent1[:crossover_point]
        child2_routes = parent2[:crossover_point]

        child1_customers = set(customer for _, route in child1_routes for customer in route)
        child2_customers = set(customer for _, route in child2_routes for customer in route)

        # Add remaining routes from other parent to ensure no duplicate customers
        for vehicle_type, route in parent2[crossover_point:]:
            new_route = [depot]
            for customer in route[1:-1]:
                if customer not in child1_customers:
                    new_route.append(customer)
                    child1_customers.add(customer)
            new_route.append(depot)
            child1_routes.append((vehicle_type, new_route))

        for vehicle_type, route in parent1[crossover_point:]:
            new_route = [depot]
            for customer in route[1:-1]:
                if customer not in child2_customers:
                    new_route.append(customer)
                    child2_customers.add(customer)
            new_route.append(depot)
            child2_routes.append((vehicle_type, new_route))

        return [child1_routes, child2_routes]
    else:
        return [parent1, parent2]

# Repairs offspring solutions by removing duplicates and inserting unassigned customers into routes.

In [14]:
def repair_routes(offspring):
    all_customers = set((cust[0], cust[1]) for cust in customers)
    customer_details = { (cust[0], cust[1]): cust for cust in customers }

    # Track customers in routes and identify duplicates
    customer_usage = {}
    for vehicle_type, route in offspring:
        for customer in route[1:-1]:  # Skip depot entries
            if (customer[0], customer[1]) in customer_usage:
                customer_usage[(customer[0], customer[1])].append((vehicle_type, route))
            else:
                customer_usage[(customer[0], customer[1])] = [(vehicle_type, route)]

    # Remove duplicates and prepared list of unassigned customers
    unassigned_customers = list(all_customers)
    for customer, routes in customer_usage.items():
        if len(routes) > 1:  # More than one occurrence means duplicates
            # Keep the first, remove from others
            for vehicle_type, route in routes[1:]:
                route.remove(customer_details[customer])
        unassigned_customers.remove(customer)

    random.shuffle(unassigned_customers)

    # Insert random unassigned customers into routes with sufficient capacity
    for customer_coords in unassigned_customers:
        customer = customer_details[customer_coords]
        inserted = False
        for vehicle_type, route in offspring:
            current_capacity = sum(c[2] for c in route[1:-1])
            if current_capacity + customer[2] <= vehicle_types[vehicle_type]['capacity']:
                route.insert(-1, customer)
                inserted = True
                break
        if not inserted:
            # Find the cost-effective vehicle that can accommodate the customer
            suitable_vehicles = [(v_type, v_specs) for v_type, v_specs in vehicle_types.items() if v_specs['capacity'] >= customer[2]]
            if suitable_vehicles:
                # Select the vehicle with the lowest cost per km
                best_vehicle = min(suitable_vehicles, key=lambda v: v[1]['cost_per_km'])
                new_vehicle_type = best_vehicle[0]
                new_route = [depot, customer, depot]  # Create a new route for this customer
                offspring.append((new_vehicle_type, new_route))
                #print(f"Added new vehicle {new_vehicle_type} for customer {customer} due to insufficient capacity.")
            else:
                print(f"No vehicle found for customer {customer}.")
    return offspring

# Applies swap mutation to a route, swapping the positions of two customers.

In [15]:
def swap_mutation(route):
    if len(route) > 3:  # Ensure there are at least two customers to swap
        idx1, idx2 = random.sample(range(1, len(route)-1), 2)
        route[idx1], route[idx2] = route[idx2], route[idx1]
    return route

# Genetic Algorithm Implementation:
Initialize population.
Evaluate fitness of individuals.
Select top-performing individuals for the next generation.
Perform crossover and mutation to create offspring.
Update population.
Repeat for a specified number of generations.

In [42]:
def genetic_algorithm(pop_size, num_vehicles, num_generations):
    population = generate_initial_population(pop_size, num_vehicles)
    for generation in range(num_generations):
        # Evaluate fitness
        fitness_scores = [fitness_function(individual) for individual in population]
        # Selection (top half)
        sorted_population = [x for _, x in sorted(zip(fitness_scores, population))]
        selected = sorted_population[:len(sorted_population)//2]

        # Crossover
        offspring = []
        while len(offspring) < len(selected):
            parent1, parent2 = random.sample(selected, 2)
            children = one_point_crossover(parent1, parent2)
            for child in children:
                repaired_child = repair_routes(child)
                offspring.append(repaired_child)

        # Mutation
        for individual in offspring:
            for vehicle_type, route in individual:
                if random.random() < 0.1:
                    swap_mutation(route)

        # Create new population
        population = selected + offspring[:len(population) - len(selected)]
        best_solution = min(population, key=fitness_function)
        best_cost = fitness_function(best_solution)

    best_solution = min(population, key=fitness_function)
    return best_solution

# To find the optimum generation and population size

In [44]:
population_sizes = [50, 100, 150, 200, 250, 300]
generation_sizes = [50, 100, 150, 200, 250, 300]

lowest_costs = {}

for pop_size in population_sizes:
    for gen_size in generation_sizes:
        best_solution = genetic_algorithm(pop_size, 5, gen_size)
        total_cost = fitness_function(best_solution)

        lowest_costs[(pop_size, gen_size)] = total_cost

best_parameters = min(lowest_costs, key=lowest_costs.get)
lowest_cost = lowest_costs[best_parameters]

print("Best Parameters:", best_parameters)
print("Lowest Cost:", lowest_cost)

Best Parameters: (200, 100)
Lowest Cost: 125.95791551618376


# Genetic algorithm Execution


In [46]:
best_solution = genetic_algorithm(200, 5, 100)

total_distance = sum(calculate_distance(route[i], route[i+1]) for vehicle_type, route in best_solution for i in range(len(route)-1))
total_distance = round(total_distance, 2)
total_cost = fitness_function(best_solution)
total_cost = round(total_cost, 2)
print(f"Total Distance: {total_distance} km")
print(f"Total Cost: RM {total_cost}\n")

vehicle_count = 1
for vehicle_type, route in best_solution:
    route_distance = sum(calculate_distance(route[i], route[i+1]) for i in range(len(route)-1))
    route_cost = route_distance * vehicle_types[vehicle_type]['cost_per_km']
    route_demand = sum(cust[2] for cust in route[1:-1])
    route_distance = round(route_distance, 3)
    route_cost = round(route_cost, 2)

    print(f"Vehicle {vehicle_count} (Type {vehicle_type}):")
    print(f"Round Trip Distance: {route_distance} km, Cost: RM {route_cost}, Demand: {route_demand}")

    route_details = "Depot"
    for i in range(1, len(route)-1):
        distance_to_next = calculate_distance(route[i], route[i+1])
        distance_to_next = round(distance_to_next, 3)
        route_details += f" -> C{customers.index(route[i])+1} ({distance_to_next} km)"
    distance_to_depot = calculate_distance(route[-2], depot)
    distance_to_depot = round(distance_to_depot, 3)
    route_details += f" -> Depot ({distance_to_depot} km)"
    print(route_details + "\n")

    vehicle_count += 1

Total Distance: 108.46 km
Total Cost: RM 130.16

Vehicle 1 (Type A):
Round Trip Distance: 28.759 km, Cost: RM 34.51, Demand: 24
Depot -> C2 (1.604 km) -> C5 (2.594 km) -> C6 (8.982 km) -> C7 (6.508 km) -> Depot (6.508 km)

Vehicle 2 (Type A):
Round Trip Distance: 62.834 km, Cost: RM 75.4, Demand: 25
Depot -> C9 (11.358 km) -> C8 (14.29 km) -> C3 (8.323 km) -> C4 (4.021 km) -> C1 (13.152 km) -> Depot (13.152 km)

Vehicle 3 (Type A):
Round Trip Distance: 16.871 km, Cost: RM 20.25, Demand: 8
Depot -> C10 (8.436 km) -> Depot (8.436 km)

